In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [5]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'


In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

OpenAI API Key exists and begins sk-proj-


In [10]:
system_message = "You are a helpful assistant in a rice store. You should try to gently encourage \
the customer to try items that are on sale. Brown are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a Brown rice', \
you could reply something like, 'Wonderful - we have lots of rice - including several that are part of our sales event.'\
Encourage the customer to buy Brown rice if they are unsure what to get."

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [13]:
system_message += "\nIf the customer asks for Indaryani rice, you should respond that shoes are not on sale today, \
but remind the customer to look at Brown rice!"

In [19]:
#multishot prompting
def chat(message, history):

    relevant_system_message = system_message
    keywords = ['discount', 'offer', 'promotion']  # Define words that imply customer is looking for a better deal

    if 'rice' not in message.strip().lower():
        relevant_system_message += (
            " The store does not sell wheat; if you are asked for wheat, be sure to point out other items on sale."
        )
    elif any(word in message.strip().lower() for word in keywords):
        relevant_system_message += (
            " If the customer asks for more money off the selling price, the store is currently running a 'buy 2 get one free' campaign, so be sure to mention this."
        )

    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [18]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
